In [17]:
import easyocr

from PIL import Image

import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt

import numpy as np

from enum import IntEnum

import cv2
import os
import json

In [18]:
!copy C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\train.jsonl C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\train_ocr.jsonl
!copy C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\test.jsonl C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\test_ocr.jsonl

‘Є®ЇЁа®ў ­® д ©«®ў:         1.
‘Є®ЇЁа®ў ­® д ©«®ў:         1.


In [19]:
data_dir = r'C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\\'

train_path = data_dir + 'train_ocr.jsonl'
dev_path = data_dir + 'test_ocr.jsonl'

train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(dev_path, lines=True)

In [20]:
train_data['path'] = train_data['img'].apply(lambda x: os.path.join(data_dir, x))
train_data

,label,img,path
0,3,img/10910387.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
1,3,img/10747369.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
2,0,img/13996726.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
3,1,img/9987066.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
4,0,img/16702250.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
...,...,...,...
626,3,img/14293988.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
627,3,img/13082683.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
628,0,img/15659398.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
629,0,img/12081951.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...


In [21]:
test_data['path'] = test_data['img'].apply(lambda x: os.path.join(data_dir, x))
test_data

,label,img,path
0,0,img/13037928.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
1,6,img/9664522.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
2,1,img/15448302.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
3,0,img/16702294.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
4,0,img/15244154.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
...,...,...,...
206,1,img/12954537.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
207,6,img/16180208.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
208,0,img/15244043.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
209,0,img/14993032.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...


In [22]:
class ImageTransformType(IntEnum):
    No = 0
    Gray = 1
    Binary = 2
    BinaryInv = 3
    AdaptiveMean = 4
    AdaptiveMeanInv = 5
    AdaptiveGaussian = 6
    AdaptiveGaussianInv = 7

In [23]:
def recognize(path:list, recognizer_name: str, image_transform: ImageTransformType, thresh: int, maxval: int, 
              block_size:int, c:float) -> list:
    recognized_text = []    
    for i in range(len(path)):
        #print(path[i])
        image = cv2.imread(path[i], cv2.COLOR_BGR2RGB)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        if image_transform == ImageTransformType.Gray:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
        if image_transform == ImageTransformType.Binary:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, image = cv2.threshold(image, thresh, maxval, cv2.THRESH_BINARY)
            
        if image_transform == ImageTransformType.BinaryInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, image = cv2.threshold(image, thresh, maxval, cv2.THRESH_BINARY_INV)

        if image_transform == ImageTransformType.AdaptiveMean:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, c)

        if image_transform == ImageTransformType.AdaptiveMeanInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size, c)
            
        if image_transform == ImageTransformType.AdaptiveGaussian:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, c)
            
        if image_transform == ImageTransformType.AdaptiveGaussianInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, c)
            
        if recognizer_name == 'easyocr':            
            recognized_text.append(' '.join(easyocr_reader.readtext(image, detail=0)))
            
        elif recognizer_name == 'pytesseract':
            recognized_text.append(pytesseract.image_to_string(image).replace('\n', ' '))
            
    return recognized_text


In [24]:
easyocr_reader = easyocr.Reader(['ru']) # this needs to run only once to load the model into memory

In [25]:
recognized_text_test = recognize(test_data['path'].values, 
     recognizer_name='easyocr', 
     image_transform=ImageTransformType.BinaryInv, 
     thresh=247, 
     maxval=62,
     block_size=0,
     c=0)


In [26]:
recognized_text_test

['сЕе- ИУшЕЕ ИЕКАРСТЕО ПАП НУ Я УЖЕ, ЕЫзДОРОВИЙ НЕТ: НЕ ВЫЗДОРОВИ^',
 'ПЛКЕНше #+ ааыТь МЯГИ пипа: щп8 ФыФщСЙ',
 'Она: рсе; мне болыше не нужны мужики] € эгого дня я гордая и независимая] Она; через #и весенним обострением кому сейчас лехко',
 'Измени свою жилнь (РИстИанствовё хорошо хоть в жопу никого не выебали впрочем еще невечер сегодия ₽ 0 05',
 'Невидимый кЕ@е[ и Хэнк Всадннки ЯП Апокалипснса Муди:',
 '',
 "' % ::' 17 ; #4 6 Мы Хотим УГоМоНИ СВОЙ ПЫл! Я не сМОПУ равенства СТОЛЬКО грабить насиловать",
 'Шуетвоим #Гинышния лааше ПммииУиыя пынэл Басаынэ Пмашаамаа Патиао @аквашынталлы @вш! ПрБаШшем',
 'СОсЕДИ ОЦЕНЯТ',
 "МЕНяАд ПРАГХЗНИК; ГАЕы 6АА; НАЩЕМУ @ЛУ; СХОАИ ЛЕБЯ ИСКАА  [ВЕКИНЬ ЕГО} ПОЖААИОТА; 'опрос}",
 '',
 '\'ЕЕРаДШ Са$  "Шаы [Шн8 ## 5: : : {\'1 { } 1. : ;! . Принеси мне ее голову!',
 '',
 '',
 '8223 (5.Ы Н4! (ОГ ШМ!',
 'ЕФТЬ [ФШРФФЫ; #Я МФТОРЫЕ ШУМШЕНЕЗМШЫШВЫ',
 '',
 '1',
 '',
 'ДИЗМЙМ ШФТШШНМ НИИШШЕ',
 'ПЩРЛ ШБДЫЯТЬ ПЛИН [ #ЩЕН МЁШПШФШМ',
 "'}: !й",
 'ВЕЧЕППННП ШЛММРЛВЕЛЕ

In [27]:

dev_rows = pd.read_json(dev_path, lines=True)
with open(data_dir + 'test_ocr.jsonl', "w", encoding="utf-8") as rf:
    for i, (_, row) in enumerate(dev_rows.iterrows()):
        dictRow = dict(row)
        dictRow["recognized_text_easyocr"] = recognized_text_test[i]
        rf.write(json.dumps(dictRow) + "\n")
        
        

In [28]:
recognized_text_train = recognize(train_data['path'].values, 
     recognizer_name='easyocr', 
     image_transform=ImageTransformType.BinaryInv, 
     thresh=247, 
     maxval=62,
     block_size=0,
     c=0)



In [29]:

train_rows = pd.read_json(train_path, lines=True)
with open(data_dir + 'train_ocr.jsonl', "w", encoding="utf-8") as rf:
    for i, (_, row) in enumerate(train_rows.iterrows()):
        dictRow = dict(row)
        dictRow["recognized_text_easyocr"] = recognized_text_train[i]
        rf.write(json.dumps(dictRow) + "\n")

In [30]:
train_path_ocr = data_dir + 'train_ocr.jsonl'
dev_path_ocr = data_dir + 'test_ocr.jsonl'

train_data_ocr = pd.read_json(train_path_ocr, lines=True)
test_data_ocr = pd.read_json(dev_path_ocr, lines=True)

In [31]:
train_data_ocr

,label,img,recognized_text_easyocr
0,3,img/10910387.jpg,Бг]~яЁЦШ ВваЗКЕ;а ~БецЕаноляЕБа #д ГеияТЪ_Цы ш...
1,3,img/10747369.jpg,[ы КУбШШЬ? ТШЛЬКШ КЛЁДАМУЕЮ: # -Ш
2,0,img/13996726.jpg,ОКАЗАЛОСЬ МОЯ СЕСТРА ПО НОЧАМ ЗАНИМАЕТСЯ ПРОСТ...
3,1,img/9987066.jpg,
4,0,img/16702250.jpg,ВЕМШДШЧЮИ ЩЛ ЛАЙК ПШВТПВШЮ Е#я
...,...,...,...
626,3,img/14293988.jpg,МШТЯП Ктьйе ПОДШЕ ПДИШЛМ
627,3,img/13082683.jpg,"СКОЛЬКО ВЕСИТ ИДЕАЛЬНЫЙ ЕВРЕЙ? 0,2 КИЛОГРАММА ..."
628,0,img/15659398.jpg,
629,0,img/12081951.jpg,"~""Яе эяю @елъ_у_~__ [ @едеек Ы₽"


In [32]:
test_data_ocr

,label,img,recognized_text_easyocr
0,0,img/13037928.jpg,"сЕе- ИУшЕЕ ИЕКАРСТЕО ПАП НУ Я УЖЕ, ЕЫзДОРОВИЙ ..."
1,6,img/9664522.jpg,ПЛКЕНше #+ ааыТь МЯГИ пипа: щп8 ФыФщСЙ
2,1,img/15448302.jpg,Она: рсе; мне болыше не нужны мужики] € эгого ...
3,0,img/16702294.jpg,Измени свою жилнь (РИстИанствовё хорошо хоть в...
4,0,img/15244154.jpg,Невидимый кЕ@е[ и Хэнк Всадннки ЯП Апокалипснс...
...,...,...,...
206,1,img/12954537.jpg,"""25 :""5й@4?й; #""; 4; ЕЕМ4И '415д - ""44 < ЁЖ5{4..."
207,6,img/16180208.jpg,
208,0,img/15244043.jpg,"Да чТа { @Жет бть кужсе, чем несколько детей М..."
209,0,img/14993032.jpg,КГДАЛРИ ЧИСКЕ МАНДАРШНМ #ЕЧЫВСАВЧЕШЬ ЗИПАХА ЮВ...
